In [1]:
#!pip install beautifulsoup4

In [2]:
import requests
import re
from bs4 import BeautifulSoup


In [3]:
infectiousdiseases = 'https://medlineplus.gov/infections.html'

try:
    response = requests.get(infectiousdiseases)
    response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
except requests.exceptions.RequestException as e:
    print('Erro ao fazer a requisição:', e)
    exit()

infectiousdiseases_html = BeautifulSoup(response.content, 'html.parser')

In [4]:
# Salvar o conteúdo HTML em um arquivo
infectiousdiseases = str(infectiousdiseases_html)
with open('infectiousdiseases.html', 'w', encoding='utf-8') as file:
    file.write(infectiousdiseases)

print('HTML salvo com sucesso no arquivo "pagina.html"')

HTML salvo com sucesso no arquivo "pagina.html"


In [5]:
# Carregar o arquivo HTML local
with open('infectiousdiseases.html', 'r', encoding='utf-8') as file:
    infectiousdiseases_reader = file.read()

# Criar um objeto BeautifulSoup
infectiousdiseases_soup = BeautifulSoup(infectiousdiseases_reader, 'html.parser')


In [6]:
# Encontrar todos os elementos com a classe "item"
items = infectiousdiseases_soup.find_all(class_='item')

# Inicializar uma lista para armazenar as doenças
diseases = []

# Exibir os textos dos elementos <a> dentro dos elementos com a classe "item"
for item in items:
    # Encontrar todos os elementos <a> dentro do elemento com a classe "item"
    a_tags = item.find_all('a')
    for a_tag in a_tags:
        disease = a_tag.text.strip()
        if disease != 'Animal Diseases and Your Health':
            diseases.append(disease)



In [7]:
# Dicionário para armazenar as doenças e sintomas
disease_symptoms = {}  
for disease in diseases:
    disease_clear = re.sub("[^\w]", "", disease.lower())
    disease_html = f'https://medlineplus.gov/{disease_clear}.html'
    
    try:
        response = requests.get(disease_html)
        response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
    except requests.exceptions.RequestException as e:
        print('Erro ao fazer a requisição:', e)
        exit()
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar o primeiro elemento <h3>
    first_h3_element = soup.find('h3', string=re.compile(r'^What are the symptoms of'))

    # Se não houver primeiro elemento <h3>, sair do loop
    if not first_h3_element:
        symptoms = []
        continue
    
    # Encontrar o próximo elemento <h3>
    next_h3_element = first_h3_element.find_next_sibling('h3')

    # Inicializar uma lista para armazenar os elementos <li>
    symptoms = []
    
    # Define o primeiro elemento <h3>
    current_element = first_h3_element

    # Enquanto não chegar ao próximo elemento <h3>
    while current_element != next_h3_element:
        # Avança para o próximo elemento irmão
        current_element = current_element.find_next_sibling()
        elements = current_element.find_all('li')
        # Verifica se o elemento é uma tag <li>
        if elements:
            # Adiciona cada item da lista de sintomas ao resultado final
            for element in elements:
                symptoms.extend([item.strip() for item in element.text.splitlines() if item.strip()])
            
    # Adiciona a doença e seus sintomas ao dicionário
    disease_symptoms[disease] = symptoms

In [19]:
# Lista para armazenar as chaves com valores vazios
empty_symptoms_keys = []

# Itera sobre as chaves do dicionário
for key, value in disease_symptoms.items():
    if not value:  # Verifica se o valor é uma lista vazia
        empty_symptoms_keys.append(key)  # Adiciona a chave à lista

print(empty_symptoms_keys)

['Body Lice', 'Infectious Diseases', 'HPV', 'Skin Infections', 'Pubic Lice']


In [33]:
# Cria um conjunto das chaves do dicionário
disease_keys_list = list(disease_symptoms.keys())

# Encontrar os itens da lista que não correspondem a uma chave no dicionário
items_not_in_dict = [disease for disease in diseases if disease not in disease_keys_list]

# Conta quantos itens não correspondem
count_items_not_in_dict = len(items_not_in_dict)

# Conta quantas doenças há na lista
count_disease = len(diseases)

print(f"Quantidade de itens da lista que não correspondem a uma chave: {count_items_not_in_dict} de um total: {count_disease}.")


Quantidade de itens da lista que não correspondem a uma chave: 186 de um total: 280.
